1. Setup envivronment

In [ ]:
## This is a Juptyer notebook for the Kaggle Project: Abalone Regresssion

%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn
## Import libaries

## Data analysis and wrangling
import numpy as np
import pandas as pd
import random as rnd

## Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

#ML Methods
from sklearn.model_selection import train_test_split


2. Problem identification 

Problem statement: 

This is one of the 2024 playground competitions on Kaggle. The major goal of the project is the develop a model for predicting the age of abalone from different physical measurements.

Performance indicator:
The model performanc will be checked by the Root Mean Squared Logarithmic Error between the actual and predicted "Rings".

Reference:
Walter Reade, Ashley Chow. (2024). Regression with an Abalone Dataset. Kaggle. https://kaggle.com/competitions/playground-series-s4e4

3. Reading data 

In [ ]:
## Reading data
train_df=pd.read_csv('C:\\Users\\Kai Qi Yan\\OneDrive\\文档\\GitHub\\Remote_Abalone_Regression\\Input\\train.csv')#
test_df=pd.read_csv('C:\\Users\\Kai Qi Yan\\OneDrive\\文档\\GitHub\\Remote_Abalone_Regression\\Input\\test.csv')

4. Basic data checks

4.1 The number of features

In [ ]:
# Number of columns and rows in the dataset
print(train_df.columns.values)
print(train_df.shape)

There are 10 columns in the data. 
The Ring is the target, id is the number of entries and others are features.
From the https://archive.ics.uci.edu/dataset/1/abalone, we obtained the following details of the features.
1. Sex: M,F,I(infant)
2. Length: longest shell distance in [mm]
3. Diameter: distance perpendicular to Length in [mm]
4. Height: the height with meat in the shell [mm]
5. Whole weight: the weight of the whole abalone(shell+meat) [g]
6. Shucked weight (named as Whole weight.1): the weight of meat [g]
7. Viscera_weight (named as Whole weight.2): the weight of gut after bleeding [g]
8. Shell weight = the weight of dried shell [g]
9. Rings = +1.5 gives the age in years (Done by cutting the shell through the cone, staining it, and counting the number of rings through a microscope)

In the training dataset, there are 90515 entries of data.

In [ ]:
train_df.head()

In [ ]:
test_df.head()

4.2 Data type of the features

In [ ]:
train_df.info()

In [ ]:
test_df.info()

There are 1 categorial input: Sex and others are numerical. While the target "Rings" is an integer, the inputs are all floats.

In [ ]:
# separate the numerical and categorical columns
num_cols = train_df._get_numeric_data().columns
cat_cols = list(set(train_df.columns) - set(num_cols))

4.3 Checking of missing entries and duplicates

4.3.1 Missing entries

In [ ]:
print("TrainData\n\n",train_df.isnull().sum())
print("_"*40)
print("TestData\n\n",test_df.isnull().sum())

There are no missing entries (NaN) in both datasets.

4.3.2 Duplicates

In [ ]:
print("TrainData","\n",train_df.duplicated().sum())
print("_"*40)
print("TestData","\n",test_df.duplicated().sum())

There are no duplicates in both datasets.

4.3 Feature engineering

4.3.1. Explore new features
The following new features are included in the datasets:

1. Aspect ratio: Length/Distance. This is a shape desriptor for the abalone, which roughly represent the roundness. 

2. Ellipsoid Volume: 4/3*pi*(Length/2)*(Distance/2)*(Height/2)

3. Meat/Whole abalone ratio:

4. Gut/Meat Ratio:



ref:https://imagej.net/ij/docs/guide/146-30.html#toc-Subsection-30.7


In [ ]:
# Drop unnecessary features
train_df.drop(['id'], axis=1, inplace=True)
test_df.drop(['id'], axis=1, inplace=True)

4.4 The distribution of the features

4.4.1 Distribution of numerical features

In [ ]:
# Descriptive statistics of numerical features
train_df.describe()

Note that in Height, the minimum value is 0, indicating a missing value as the height of an abalone can never be 0.
Looking at the maximum/minimum and the mean values, the large differences between the them may indicate the features are either skewed or with outliers. 
This will be check again in the distribution plot again in the later section.

In [ ]:
# plot the distribution of the feature variables
# Plot histograms of each variable
axes = train_df.hist(bins=60, figsize=(20,15), color='skyblue', edgecolor='black', alpha=0.7, layout=(len(train_df.columns),2))

# Add labels
for ax, column in zip(axes.flatten(), train_df.columns):
    ax.set_title(column)

# Improve layout
plt.tight_layout()
plt.show()

##### why the last column is missing?

From the above plots, it is observed that most of the numerical features are skewed. Also, the magnitude of the Shell weight is much larger than the other variables. Noramlization and standardization maybe required.

In [ ]:
test_df.describe()

In [ ]:
# plot the distribution of the feature variables
# Plot histograms of each variable
axes = test_df.hist(bins=60, figsize=(20,15), color='skyblue', edgecolor='black', alpha=0.7, layout=(len(test_df.columns), 2))

# Add labels
for ax, column in zip(axes.flatten(), test_df.columns):
    ax.set_title(column)

# Improve layout
plt.tight_layout()
plt.show()

In [ ]:
# Check the number of missing values in height column
print("The number of 0 in the Height column of the train dataset is:",(train_df.Height==0).sum())
print("The number of 0 in the Height column of the test dataset is:",(test_df.Height==0).sum())

In the test dataset, there are also missing value (0) in the Height column. Before the prediction, we will need to handle them.

4.4.2 Distribution of categorial features

In [ ]:
# Descriptive statistics of categorical features
train_df.describe(include=['O'])

In [ ]:
test_df.describe(include=['O'])

4.5 indentify outliers